#Sentimental Analysis using Opinion Target and Opinion Words.
#This project performed sentimental analysis based on opinion words (like good, bad, beautiful, wrong, best, awesome, etc) of #selected opinion target ( like product name for amazon product reviews). 

##conducting for 300 reviews


In [1]:
pip install -U textblob


Requirement already up-to-date: textblob in c:\users\dell\anaconda3\lib\site-packages (0.15.3)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from collections import defaultdict 
from textblob import TextBlob
import re


In [4]:
# data preprocessing


def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r" v", " very", phrase)
    return phrase

reviewerID =[]
productID = []
reviewerName=[]
liked_and_seen = []
reviewText = []
rating = []
summary = []
unixTime = []
date = []

with open('Cell_Phones_Accessories.json') as json_data:
    d = json.load(json_data)
    d = d[0:300]

for i in range(len(d)):
    reviewText.append(decontracted(d[i]['reviewText']))
    rating.append(d[i]['overall'])
    reviewerID.append(d[i]['reviewerID'])
    productID.append(d[i]['asin'])
#    reviewerName.append(d[i]['reviewerName'])
    liked_and_seen.append(d[i]['helpful'])
    summary.append(d[i]['summary'])
    unixTime.append(d[i]['unixReviewTime'])
    date.append(d[i]['reviewTime'])

In [5]:
from pandas import DataFrame
dataset = DataFrame({'reviewerID': reviewerID, 'productID': productID, 'liked_and_seen': liked_and_seen, 'reviewText': reviewText, 'rating': rating, 'summary': summary, 'unixTime': unixTime, 'date': date})

In [6]:
#cleaning unwanted symbols
#cleaning unwanted symbols
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

comment_dict = defaultdict(list)
for i in range(len(dataset)):
    sentence = re.sub('[^a-zA-Z.]',' ',dataset['reviewText'][i])
    sentence = sentence.lower()
    sentence = sentence.split('.')
    for k in range(len(sentence)):
        review = sentence[k].split()
        review = [word for word in review if not word in set(stopwords.words('english'))]
        sentence[k] =  ' '.join(review)
        comment_dict[i].append(sentence[k])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#delete unwanted '' words
for j in range(len(comment_dict)):
    comment_dict[j] = [comment_dict[j][i] for i in range(len(comment_dict[j])) if comment_dict[j][i] not in '']


In [8]:
for i in range(len(comment_dict)):
    reviewText[i] = ('. '.join(comment_dict[i][j] for j in range(len(comment_dict[i]))))

In [9]:
# spelling correction
for i in range(len(reviewText)):
    b = TextBlob(reviewText[i])
    reviewText[i] = b.correct()

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [13]:
# creating corpus
corpus = defaultdict(set)
for i in range(len(reviewText)):
    wiki = reviewText[i]
    corpus[i] = wiki.sentences

In [14]:
corpus_key = corpus.keys()
corpus_list = defaultdict(list)

for i in corpus_key:
    for j in range(len(corpus[i])):
        word = ' '.join(corpus[i][j].words)
        corpus_list[i].append(word)

In [15]:
# dding biwords and triwords for generating patterns

length = defaultdict(list)
for i in corpus_key:
    length[i] = list(corpus_list[i])

In [16]:
# triwords       
for i in corpus_key:
    for j in range(len(length[i])):
        text = TextBlob(length[i][j])
        text = text.ngrams(n=3)
        for k in range(len(text)):
            triword = [' '.join([text[k][l] for l in range(len(text[k]))])]
            triword = triword[0]
            corpus_list[i].append(triword)

In [17]:
#biwords
for i in corpus_key:
    for j in range(len(length[i])):
        text = TextBlob(length[i][j])
        text = text.ngrams(n=2)
        for k in range(len(text)):
            triword = [' '.join([text[k][l] for l in range(len(text[k]))])]
            triword = triword[0]
            corpus_list[i].append(triword)
                  

In [19]:
 
import nltk
nltk.download('averaged_perceptron_tagger')

        


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [20]:
# Part-of-speech Tagging
pos_dict = defaultdict(list)
for i in corpus_key:
    for j in range(len(corpus_list[i])):
        text = TextBlob(corpus_list[i][j])
        text = text.tags
        pos_dict[i].append(text)

pos_dict_key = pos_dict.keys()

corpus_noun = defaultdict(list)
for i in pos_dict_key:
    for j in range(len(pos_dict[i])):
        for k in range(len(pos_dict[i][j])):
            if(pos_dict[i][j][k][1] == 'NN'):
                corpus_noun[i].append(pos_dict[i][j][k])  

In [21]:
# pattern generation from Part -of -speech tagging
pattern1 =  defaultdict(list)
for i in pos_dict_key:
    for j in range(len(pos_dict[i])):
        if(len(pos_dict[i][j]) == 2):
            if((pos_dict[i][j][0][1] == 'JJ' and pos_dict[i][j][1][1] == 'NN') or (pos_dict[i][j][0][1] == 'JJ' and pos_dict[i][j][1][1] == 'NNS')):
                #pattern1  
                pattern1[i].append(pos_dict[i][j])

In [22]:
pattern2 =  defaultdict(list)
for i in pos_dict_key:
    for j in range(len(pos_dict[i])):          
        if(len(pos_dict[i][j]) == 3):
            if((pos_dict[i][j][0][1] == 'JJ' and pos_dict[i][j][1][1] == 'NN'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'JJ' and pos_dict[i][j][1][1] == 'NN'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern2
                pattern2[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'JJ' and pos_dict[i][j][1][1] == 'NNS'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'JJ' and pos_dict[i][j][1][1] == 'NNS'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern2
                pattern2[i].append(pos_dict[i][j])
                   

In [23]:
pattern3 =  defaultdict(list)
for i in pos_dict_key:
    for j in range(len(pos_dict[i])):
        if(len(pos_dict[i][j]) == 2):
            if((pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'JJ') or (pos_dict[i][j][0][1] == 'RBR' and pos_dict[i][j][1][1] == 'JJ') or (pos_dict[i][j][0][1] == 'RBS' and pos_dict[i][j][1][1] == 'JJ')):
                #pattern3
                pattern3[i].append(pos_dict[i][j]) 

In [24]:
pattern4 =  defaultdict(list)
for i in pos_dict_key:
    for j in range(len(pos_dict[i])):          
        if(len(pos_dict[i][j]) == 3):
            if((pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'JJ'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'JJ'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RB'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RB'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RBR'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RBR'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RBS'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RBS'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RBR' and pos_dict[i][j][1][1] == 'JJ'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'JJ'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RBR' and pos_dict[i][j][1][1] == 'RB'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RB'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RBR' and pos_dict[i][j][1][1] == 'RBR'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RBR'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RBR' and pos_dict[i][j][1][1] == 'RBS'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RBS'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RBS' and pos_dict[i][j][1][1] == 'JJ'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'JJ'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RBS' and pos_dict[i][j][1][1] == 'RB'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RB'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RBS' and pos_dict[i][j][1][1] == 'RBR'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RBR'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RBS' and pos_dict[i][j][1][1] == 'RBS'and pos_dict[i][j][2][1] == 'NN') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RBS'and pos_dict[i][j][2][1] == 'NNS')):
                #pattern4
                pattern4[i].append(pos_dict[i][j])

In [25]:
pattern5 =  defaultdict(list)
for i in pos_dict_key:
    for j in range(len(pos_dict[i])):
        if(len(pos_dict[i][j]) == 2):
            if((pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'VBN') or (pos_dict[i][j][0][1] == 'RBR' and pos_dict[i][j][1][1] == 'VBN') or (pos_dict[i][j][0][1] == 'RBS' and pos_dict[i][j][1][1] == 'VBN')):
                #pattern5
                pattern5[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'VBD') or (pos_dict[i][j][0][1] == 'RBR' and pos_dict[i][j][1][1] == 'VBD') or (pos_dict[i][j][0][1] == 'RBS' and pos_dict[i][j][1][1] == 'VBD')):
                #pattern5
                pattern5[i].append(pos_dict[i][j])

In [26]:
pattern6 =  defaultdict(list)
for i in pos_dict_key:
    for j in range(len(pos_dict[i])):          
        if(len(pos_dict[i][j]) == 3):
            if((pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RB'and pos_dict[i][j][2][1] == 'JJ') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RBR'and pos_dict[i][j][2][1] == 'JJ') or (pos_dict[i][j][0][1] == 'RB' and pos_dict[i][j][1][1] == 'RBS'and pos_dict[i][j][0][1] == 'JJ')):
                #pattern6
                pattern6[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RBR' and pos_dict[i][j][1][1] == 'RB'and pos_dict[i][j][2][1] == 'JJ') or (pos_dict[i][j][0][1] == 'RBR' and pos_dict[i][j][1][1] == 'RBR'and pos_dict[i][j][2][1] == 'JJ') or (pos_dict[i][j][0][1] == 'RBR' and pos_dict[i][j][1][1] == 'RBS'and pos_dict[i][j][0][1] == 'JJ')):
                #pattern6
                pattern6[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'RBS' and pos_dict[i][j][1][1] == 'RB'and pos_dict[i][j][2][1] == 'JJ') or (pos_dict[i][j][0][1] == 'RBS' and pos_dict[i][j][1][1] == 'RBR'and pos_dict[i][j][2][1] == 'JJ') or (pos_dict[i][j][0][1] == 'RBS' and pos_dict[i][j][1][1] == 'RBS'and pos_dict[i][j][0][1] == 'JJ')):
                #pattern6
                pattern6[i].append(pos_dict[i][j])


In [27]:
pattern7 =  defaultdict(list)
for i in pos_dict_key:
    for j in range(len(pos_dict[i])):
        if(len(pos_dict[i][j]) == 2):
            if((pos_dict[i][j][0][1] == 'VBN' and pos_dict[i][j][1][1] == 'NN') or (pos_dict[i][j][0][1] == 'VBD' and pos_dict[i][j][1][1] == 'NN')):
                #pattern7
                pattern7[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'VBN' and pos_dict[i][j][1][1] == 'NNS') or (pos_dict[i][j][0][1] == 'VBD' and pos_dict[i][j][1][1] == 'NNS')):
                #pattern7
                pattern7[i].append(pos_dict[i][j])
                

In [28]:
pattern8 =  defaultdict(list)
for i in pos_dict_key:
    for j in range(len(pos_dict[i])):
        if(len(pos_dict[i][j]) == 2):
            if((pos_dict[i][j][0][1] == 'VBN' and pos_dict[i][j][1][1] == 'RB') or (pos_dict[i][j][0][1] == 'VBD' and pos_dict[i][j][1][1] == 'RB')):
                #pattern8
                pattern8[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'VBN' and pos_dict[i][j][1][1] == 'RBR') or (pos_dict[i][j][0][1] == 'VBD' and pos_dict[i][j][1][1] == 'RBR')):
                #pattern8
                pattern8[i].append(pos_dict[i][j])
            elif((pos_dict[i][j][0][1] == 'VBN' and pos_dict[i][j][1][1] == 'RBS') or (pos_dict[i][j][0][1] == 'VBD' and pos_dict[i][j][1][1] == 'RBS')):
                #pattern8
                pattern8[i].append(pos_dict[i][j])

In [29]:
pattern = defaultdict(set)
pattern.update(pattern1)
pattern.update(pattern2)
pattern.update(pattern3)
pattern.update(pattern4)
pattern.update(pattern5)
pattern.update(pattern6)
pattern.update(pattern7)
pattern.update(pattern8)

In [30]:
#select stuff from OT_OW_key
# Semi- supervised approach creates Opinion Target from stuff.

stuff = ['software', 'application', 'service', 'power supply', 'sim card', 'display', 
         'storage space', 'sensor', 'wireless charging', 'design', 'cpu', 'accessories',
         'camera','quality','time','condition','screen','price','case','build','access',
         'battery','buy','power','switch','light','design','technology','radio','fashion'
'product','charging','feature','touch','profile','car','slot','tables','construction',
'period ','system','game','bottom','sound','blackberry charge','price anyone','price extra',
'cord length','charge port',' phone','horizon charge','fraction price','charge ','key',
'extension','internet','cheap','cover','speaker']

In [32]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [33]:
#Finding Similar waords of above Opinion Targets

from nltk.corpus import wordnet as wn
from itertools import chain

stuff_OT = defaultdict(set)
OT2 = set()

synsets_set = defaultdict(set)
hyponyms_set = defaultdict(set)
for z in range(len(stuff)): 
    input_word = stuff[z] 
    OT1= set()    
    for i,j in enumerate(wn.synsets(input_word)):
     
        
        hypernyms = ', '.join(list(chain(*[l.lemma_names() for l in j.hypernyms()])))
   
        synsets_set[i].add(hypernyms)
        
        hyponyms = ', '.join(list(chain(*[l.lemma_names() for l in j.hyponyms()])))
    
        hyponyms_set[i].add(hyponyms)
    

        ho = [hypernyms]
        for h in range(len(ho)):
            temp_list = ho[h].split(', ')
            if(temp_list != ['']):
                for l in range(len(temp_list)):
                    temp_word = ' '.join(temp_list[l].split('_'))
                    OT2.add(temp_word)
                    OT1.add(temp_word)
        
        hy = [hypernyms]
        for h in range(len(hy)):
            temp_list = hy[h].split(', ')
            if(temp_list != ['']):
                for l in range(len(temp_list)):
                    temp_word = ' '.join(temp_list[l].split('_'))
                    OT2.add(temp_word)
                    OT1.add(temp_word)
    OT1 = list(OT1)
    for i in range(len(OT1)):
        stuff_OT[stuff[z]].add(OT1[i])


In [34]:
hyponyms_set_keys = hyponyms_set.keys()
synsets_set_keys = synsets_set.keys()

for z in range(len(stuff)):  
    
    for k in hyponyms_set_keys:
        hy = hyponyms_set[k]
        hy = list(hy)
        for h in range(len(hy)):
            temp_list = hy[h].split(', ')
            if(temp_list != ['']):
                for l in range(len(temp_list)):
                    temp_word = ' '.join(temp_list[l].split('_'))
                    OT2.add(temp_word)
                    

In [35]:
for z in range(len(stuff)):    
    for k in synsets_set_keys:
        hy = synsets_set[k]
        hy = list(hy)
        for h in range(len(hy)):
            temp_list = hy[h].split(', ')
            if(temp_list != ['']):
                for l in range(len(temp_list)):
                    temp_word = ' '.join(temp_list[l].split('_'))
                    OT2.add(temp_word)

In [36]:
OT2 = list(OT2)

list_of_subset = defaultdict(set)
for i in range(len(stuff)):
    stuff_OT[stuff[i]] = list(stuff_OT[stuff[i]])
    for j in range(len(stuff_OT[stuff[i]])):
        list_of_subset[i].add(stuff[i])
        list_of_subset[i].add(stuff_OT[stuff[i]][j])


In [37]:
for i in list_of_subset.keys():
    list_of_subset[i] = list(list_of_subset[i])
    

In [38]:
list_of_subset2 =[]
for i in range(len(stuff)):
    list_of_subset2.append(stuff[i])

for i in list_of_subset.keys():
    for j in range(len(list_of_subset[i])):
        list_of_subset2.append(list_of_subset[i][j])

stuff = list_of_subset2

In [39]:
#  Finding Opinion Words of above Opinion Target(Stuff + its similar words.)
#         from pattern generated in step 3.

OW = defaultdict(set)
OT = defaultdict(set)
OT_OW = defaultdict(set)
OW_OT = defaultdict(set)

# it has 1 OW
pattern1_OT_OW =  defaultdict(set)
pattern1_OW_OT =  defaultdict(set)
p1_keys = pattern1.keys()
for i in p1_keys:
    if( pattern1[i] != []):
        for j in range(len(pattern1[i])):
            OT[i].add(pattern1[i][j][1][0])
            OW[i].add(pattern1[i][j][0][0])
            if(pattern1[i][j][1][0] in stuff):
                OT_OW[pattern1[i][j][1][0]].add(pattern1[i][j][0][0])
                OW_OT[pattern1[i][j][0][0]].add(pattern1[i][j][1][0])
                
                pattern1_OT_OW[pattern1[i][j][1][0]].add(pattern1[i][j][0][0])
                pattern1_OW_OT[pattern1[i][j][0][0]].add(pattern1[i][j][1][0])


In [40]:
#it has 1 OW             
pattern2_OT_OW = defaultdict(set)
pattern2_OW_OT = defaultdict(set)
p2_keys = pattern2.keys()
for i in p2_keys:
    if( pattern2[i] != []):
        for j in range(len(pattern2[i])):    
            target = pattern2[i][j][1][0] + " " + pattern2[i][j][2][0]
            OT[i].add(target)
            OW[i].add(pattern2[i][j][0][0])
            if(pattern2[i][j][1][0] in stuff or pattern2[i][j][2][0] in stuff or target in stuff):
                OT_OW[target].add(pattern2[i][j][0][0])
                OW_OT[pattern2[i][j][0][0]].add(target)
            
                pattern2_OT_OW[target].add(pattern2[i][j][0][0])
                pattern2_OW_OT[pattern2[i][j][0][0]].add(target)

In [41]:
# dont filter here
# it has 2 OW pretty, good we use only pretty good combination
pattern3_OW_OT = defaultdict(set)       
p3_keys = pattern3.keys()
for i in p3_keys:
    if( pattern3[i] != []):
        for j in range(len(pattern3[i])):    
            target = pattern3[i][j][0][0] + " " + pattern3[i][j][1][0]
            OW[i].add(target)

            OW_OT[target].add('NO Opinion Target found')
            
            pattern3_OW_OT[target].add('NO Opinion Target found')

In [42]:
# we use near much, near, much word combinations in OW here
pattern4_OT_OW = defaultdict(set)         
pattern4_OW_OT = defaultdict(set)  
p4_keys = pattern4.keys()
for i in p4_keys:
    if( pattern4[i] != []):
        for j in range(len(pattern4[i])):    
            word = pattern4[i][j][0][0] + " " + pattern4[i][j][1][0] 
            OT[i].add(pattern4[i][j][2][0])
            OW[i].add(word)
            if(pattern4[i][j][2][0] in stuff):
                OT_OW[pattern4[i][j][2][0]].add(word)
                
                OW_OT[word].add(pattern4[i][j][2][0])
                
                pattern4_OT_OW[pattern4[i][j][2][0]].add(word)
                
                pattern4_OW_OT[word].add(pattern4[i][j][2][0])

In [43]:
# dont filter here            
pattern5_OW_OT = defaultdict(set)           
p5_keys = pattern5.keys()
for i in p5_keys:
    if( pattern5[i] != []):
        for j in range(len(pattern5[i])):    
            target = pattern5[i][j][0][0] + ' ' + pattern5[i][j][1][0]
            OW[i].add(target)
            OW_OT[target].add('No Opinion Target found')
            
            pattern5_OW_OT[target].add('No Opinion Target found')

In [44]:
# dont filter here
pattern6_OW_OT = defaultdict(set)
p6_keys = pattern6.keys()
for i in p6_keys:
    if( pattern6[i] != []):
        for j in range(len(pattern6[i])):    
            target = pattern6[i][j][0][0] + " " + pattern6[i][j][2][0]
            OW[i].add(target)
            OW_OT[target].add('NO Opinion Target found')
            
            pattern6_OW_OT[target].add('NO Opinion Target found')
                

In [45]:
pattern7_OW_OT = defaultdict(set)
pattern7_OT_OW = defaultdict(set)
p7_keys = pattern7.keys()
for i in p7_keys:
    if( pattern7[i] != []):
        for j in range(len(pattern7[i])):    
            target = pattern7[i][j][1][0]
            OT[i].add(target)
            OW[i].add(pattern7[i][j][0][0])
            
            if(target in stuff):
                OW_OT[pattern7[i][j][0][0]].add(target)
                OT_OW[target].add(pattern7[i][j][0][0])
                
                pattern7_OW_OT[pattern7[i][j][0][0]].add(target)
                pattern7_OT_OW[target].add(pattern7[i][j][0][0])

In [46]:
# dont filter here
pattern_8_OW_OT = defaultdict(set)
p8_keys = pattern8.keys()
for i in p8_keys:
    if( pattern8[i] != []):
        for j in range(len(pattern8[i])):    
            target = pattern8[i][j][1][0]
            
            OW[i].add(target)
            OW_OT[target].add('No Opinion Target found')
            pattern_8_OW_OT[target].add('No Opinion Target found')

In [47]:
# Step 8: Finding similar Words of above Opinion Words.
            
OW_OT_key = OW_OT.keys()
OW_list = list(OW_OT_key)

In [48]:
from nltk.corpus import wordnet as wn
from itertools import chain


In [49]:
stuff_OW = defaultdict(set)
OW2 = set()

In [53]:
synsets_set = defaultdict(set)
hyponyms_set = defaultdict(set)
for z in range(len(OW_list)): 
    input_word = OW_list[z] 
    OW1= set()    
    for i,j in enumerate(wn.synsets(input_word)):
    #    print ('Meaning', i, 'NLTK ID: ', j.name())
        
        hypernyms = ', '.join(list(chain(*[l.lemma_names() for l in j.hypernyms()])))
    #    print ('Hypernyms:', hypernyms)
        synsets_set[i].add(hypernyms)
        
        hyponyms = ', '.join(list(chain(*[l.lemma_names() for l in j.hyponyms()])))
    #    print ('Hyponyms:', hyponyms)
        hyponyms_set[i].add(hyponyms)
    #    print() 

        ho = [hypernyms]
        for h in range(len(ho)):
            temp_list = ho[h].split(', ')
            if(temp_list != ['']):
                for l in range(len(temp_list)):
                    temp_word = ' '.join(temp_list[l].split('_'))
                    OW2.add(temp_word)
                    OW1.add(temp_word)
        
        hy = [hypernyms]
        for h in range(len(hy)):
            temp_list = hy[h].split(', ')
            if(temp_list != ['']):
                for l in range(len(temp_list)):
                    temp_word = ' '.join(temp_list[l].split('_'))
                    OW2.add(temp_word)
                    OW1.add(temp_word)
    OW1 = list(OW1)
    for i in range(len(OW1)):
        stuff_OW[OW_list[z]].add(OW1[i])


In [54]:
hyponyms_set_keys = hyponyms_set.keys()
synsets_set_keys = synsets_set.keys()

In [57]:
for z in range(len(OW_list)):    
    for k in hyponyms_set_keys:
        hy = hyponyms_set[k]
        hy = list(hy)
        for h in range(len(hy)):
            temp_list = hy[h].split(', ')
            if(temp_list != ['']):
                for l in range(len(temp_list)):
                    temp_word = ' '.join(temp_list[l].split('_'))
                    OW2.add(temp_word)

In [56]:
for z in range(len(OW_list)):    
    for k in synsets_set_keys:
        hy = synsets_set[k]
        hy = list(hy)
        for h in range(len(hy)):
            temp_list = hy[h].split(', ')
            if(temp_list != ['']):
                for l in range(len(temp_list)):
                    temp_word = ' '.join(temp_list[l].split('_'))
                    OW2.add(temp_word)

In [58]:
OW_concept = []
OW2 = list(OW2)
for i in range(len(OW2)):
    if(OW2[i] != ''):
        OW_concept.append(OW2[i])
        
for i in range(len(OW_list)):
    OW_concept.append(OW_list[i])
           
# Here OW_concept creates a list of all similar Opinion words of Opinion words generated from pattern

In [59]:
# Finding Opinion Weight of above similar Opinion Words ( from testimonial.sentiment.polarity )

OW_OT = OW_concept                   
OW_OT_key = OW_OT
OT_OW_key = OT_OW.keys()

OW_in_corpus = defaultdict(set)
for i in corpus.keys():
    for j in range(len(corpus[i])):
        word = corpus[i][j].words
        for k in range(len(word)):
            if( word[k] in OW_OT_key):
              OW_in_corpus[i].add(word[k])

In [61]:
for i in OW_in_corpus.keys():
    OW_in_corpus[i] = list(OW_in_corpus[i]) 

In [64]:
testimonial_sentiment = defaultdict(list)
testimonial_sentiment_polarity = defaultdict(list)

In [65]:
for i in OW_in_corpus.keys():
    for j in range(len(OW_in_corpus[i])):
        testimonial = TextBlob(OW_in_corpus[i][j])
        testimonial_sentiment[OW_in_corpus[i][j]].append(testimonial.sentiment)
        testimonial_sentiment_polarity[OW_in_corpus[i][j]].append(testimonial.sentiment.polarity)

In [66]:
OW_in_corpus_list = defaultdict(list)
OW_in_corpus_value = defaultdict(list)
for i in OW_in_corpus.keys():
    for j in range(len(OW_in_corpus[i])):
        word = [OW_in_corpus[i][j]]
#        for k in range(len(word)):
        if(word[0] in testimonial_sentiment_polarity.keys()):
            polarity = testimonial_sentiment_polarity[word[0]]
            OW_in_corpus_value[i].append(polarity[0])
            OW_in_corpus_list[i].append(word[0])


In [67]:
dictionary1 = dict()
dictionary2 = dict()
key_value_pair = defaultdict(list)
for i in range(len(OW_in_corpus_list)):
    for j in range(len(OW_in_corpus_list[i])):
#        dictionary1[OW_in_corpus_value[i][j]] = OW_in_corpus_list[i][j]
        dictionary2[ OW_in_corpus_list[i][j]] = OW_in_corpus_value[i][j]  

In [69]:
# Creating Words and its score in tuple_word_score                

score = defaultdict(list)
words = OW_in_corpus_list
OW_in_corpus_value_key = OW_in_corpus_value.keys()
for i in OW_in_corpus_value_key:
    for j in range(len(OW_in_corpus_value[i])):
        score[i].append(OW_in_corpus_value[i][j])
                


In [70]:
                
tuple_word_score = defaultdict(list)
for i in OW_in_corpus_value_key:
    for j in range(len(OW_in_corpus_value[i])):
        tuple_word_score[i].append((words[i][j], score[i][j]))
        
Opinion_Words = tuple_word_score

list_Opinion_Words = []
for i in range(len(Opinion_Words)):
    list_Opinion_Words.append(Opinion_Words[i])

In [76]:
#Finding Maximum scored words.                

max_abs_score = defaultdict(list)
for i in range(len(tuple_word_score)):
    maxx = 0
    for j in range(len(tuple_word_score[i])):
        temp = abs(tuple_word_score[i][j][1])
        if(temp > maxx):
            maxx = abs(tuple_word_score[i][j][1])
            maxx_word = tuple_word_score[i][j][0]
    max_abs_score[i].append((maxx_word, maxx))

In [77]:
maxx_OW_value = defaultdict(set)
for i in range(len(max_abs_score)):
    maxx_OW_value[max_abs_score[i][0][0]].add(max_abs_score[i][0][1])
    
New_OW = maxx_OW_value.keys()

In [78]:
#  Collecting important opinion words.

list_max_abs_score = []
for i in range(len(max_abs_score)):
    list_max_abs_score.append(max_abs_score[i])

In [79]:
#  Searching it again

OW_in_corpus2 = defaultdict(set)
for i in range(len(corpus)):
    for j in range(len(corpus[i])):
        word = corpus[i][j].words
        for k in range(len(word)):
            if( word[k] in New_OW):
              OW_in_corpus2[i].add(word[k])
              

In [80]:
for i in OW_in_corpus2.keys():
    OW_in_corpus2[i] = list(OW_in_corpus2[i])            
              

In [81]:
testimonial_sentiment = defaultdict(list)
testimonial_sentiment_polarity = defaultdict(list)

In [82]:
for i in OW_in_corpus2.keys():
    for j in range(len(OW_in_corpus2[i])):
        testimonial = TextBlob(OW_in_corpus2[i][j])
        testimonial_sentiment[OW_in_corpus2[i][j]].append(testimonial.sentiment)
        testimonial_sentiment_polarity[OW_in_corpus2[i][j]].append(testimonial.sentiment.polarity)


In [93]:
i

299

In [83]:
OW_in_corpus_list = defaultdict(list)
OW_in_corpus_value = defaultdict(list)
for i in OW_in_corpus2.keys():
    for j in range(len(OW_in_corpus2[i])):
        word = [OW_in_corpus2[i][j]]
#        for k in range(len(word)):
        if(word[0] in testimonial_sentiment_polarity.keys()):
            polarity = testimonial_sentiment_polarity[word[0]]
            OW_in_corpus_value[i].append(polarity[0])
            OW_in_corpus_list[i].append(word[0])


In [84]:
dictionary1 = dict()
dictionary2 = dict()
key_value_pair = defaultdict(list)
for i in range(len(OW_in_corpus_list)):
    for j in range(len(OW_in_corpus_list[i])):
#        dictionary1[OW_in_corpus_value[i][j]] = OW_in_corpus_list[i][j]
        dictionary2[ OW_in_corpus_list[i][j]] = OW_in_corpus_value[i][j]  


In [85]:
import numpy as np
              
score = defaultdict(list)
words = OW_in_corpus_list
OW_in_corpus_value_key = OW_in_corpus_value.keys()
for i in OW_in_corpus_value_key:
    for j in range(len(OW_in_corpus_value[i])):
        score[i].append(OW_in_corpus_value[i][j])
        
        
tuple_word_score = defaultdict(list)
for i in OW_in_corpus_value_key:
    for j in range(len(OW_in_corpus_value[i])):
        tuple_word_score[i].append((words[i][j], score[i][j]))

In [86]:
final_score = defaultdict(list)
for i in range(len(tuple_word_score)):
    final_score[i].append(0)
    

In [87]:
for i in range(len(tuple_word_score)):
    for j in range(len(tuple_word_score[i])):
        final_score[i].append(tuple_word_score[i][j][1])

In [92]:
i

299

In [88]:
final_score = defaultdict(list)
for i in range(len(tuple_word_score)):
    final_score[i].append(0)

In [89]:
for i in range(len(tuple_word_score)):
    for j in range(len(tuple_word_score[i])):
        final_score[i].append(tuple_word_score[i][j][1])

In [90]:
i

299

In [91]:
final_score

defaultdict(list,
            {0: [0, 0.7],
             1: [0, 0.5, 0.8],
             2: [0, 0.5, 0.8, 1.0],
             3: [0, 0.8, 0.6, 1.0, 0.4],
             4: [0, 0.8, 1.0],
             5: [0, 0.43333333333333335, 0.3],
             6: [0, 0.8],
             7: [0, 0.25],
             8: [0, 0.35, 0.7],
             9: [0, 0.5, 0.43333333333333335, 0.8],
             10: [0, 1.0],
             11: [0, 0.2, 0.6, 0.25, 0.3],
             12: [0, 0.43333333333333335, 0.5],
             13: [0, 1.0, 0.4333333333333333, 0.22727272727272727, 0.4],
             14: [0, 0.13636363636363635, 1.0],
             15: [0, 0.8],
             16: [0, 0.8, 1.0],
             17: [0, -0.5, 1.0],
             18: [0, -0.05, 0.4, 0.2, 0.16, 0.4166666666666667, -0.5],
             19: [0, 0.8, 0.25, -0.25],
             20: [0, 0.5, 0.5],
             21: [0],
             22: [0, -0.25],
             23: [0, 0.5, 0.5, 0.6],
             24: [0],
             25: [0, 0.8, -0.1875],
             

In [95]:
# Creating Average
average_score = defaultdict(list)
for i in range(len(final_score)):
    average_score[i].append(np.mean(final_score[i]))


for i in range(len(average_score)):
    if(np.isnan(average_score[i]) == True):
        average_score[i] = [0]

list_average_score = []
for i in range(len(average_score)):
    list_average_score.append(average_score[i])

In [96]:
i

299

In [97]:
import numpy as np
  
X_train = []   
for i in average_score.keys():
    temp = set()
    for j in range(len(OW_in_corpus[i])):
        temp.add(OW_in_corpus[i][j])
    X_train.append(temp)

In [98]:
j

19

In [99]:
l = set()
for i in range(len(X_train)):
    X_train[i] = list (X_train[i])
    for j in range(len(X_train[i])):
        l.add(X_train[i][j])

In [100]:
l =list(l)


In [101]:
import pandas as pd
da = pd.DataFrame(columns = l, index = OW_in_corpus.keys(), data = 0)


In [102]:
da

sturdy  tax  correctly  leave  fit  spring  wound  cap  veryirtually  \
0         0    0          0      0    0       0      0    0             0   
1         0    0          0      0    0       0      0    0             0   
2         0    0          0      0    0       0      0    0             0   
3         0    0          0      0    0       0      0    0             0   
4         0    0          0      0    0       0      0    0             0   
..      ...  ...        ...    ...  ...     ...    ...  ...           ...   
295       0    0          0      0    0       0      0    0             0   
296       0    0          0      0    0       0      0    0             0   
297       0    0          0      0    0       0      0    0             0   
298       0    0          0      0    0       0      0    0             0   
299       0    0          0      0    0       0      0    0             0   

     read  ...  blow  free  break  have  tug  wire  past  give  connected  \
0       0  ...     0     0      0     0    0     0     0     0          0   
1       0  ...     0     0      0     0    0     0     0     0          0   
2       0  ...     0     0      0     0    0     0     0     0          0   
3       0  ...     0     0      0     0    0     0     0     0          0   
4       0  ...     0     0      0     0    0     0     0     0          0   
..    ...  ...   ...   ...    ...   ...  ...   ...   ...   ...        ...   
295     0  ...     0     0      0     0    0     0     0     0          0   
296     0  ...     0     0      0     0    0     0     0     0          0   
297     0  ...     0     0      0     0    0     0     0     0          0   
298     0  ...     0     0      0     0    0     0     0     0          0   
299     0  ...     0     0      0     0    0     0     0     0          0   

     need  
0       0  
1       0  
2       0  
3       0  
4       0  
..    ...  
295     0  
296     0  
297     0  
298     0  
299     0  

[300 rows x 546 columns]

In [103]:
k = list(OW_in_corpus.keys())

In [104]:
k

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [105]:
for m in range(len(k)):
    for j in range(len(l)):
        if(l[j] in OW_in_corpus[k[m]]):
            da.iloc[m][l[j]] = 1

In [106]:
m

299

In [107]:
X_train = da.iloc[:, 0:]
X_train = np.array(X_train)

In [111]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [108]:
y_train = []
for i in average_score.keys():
    y_train.append(average_score[i][0]) 

In [112]:
y_train

array([ 0.35      ,  0.43333333,  0.575     ,  0.56      ,  0.6       ,
        0.24444444,  0.4       ,  0.125     ,  0.35      ,  0.43333333,
        0.5       ,  0.27      ,  0.31111111,  0.41212121,  0.37878788,
        0.4       ,  0.6       ,  0.16666667,  0.08952381,  0.2       ,
        0.33333333,  0.        , -0.125     ,  0.4       ,  0.        ,
        0.20416667, -0.11666667,  0.30416667,  0.        ,  0.4       ,
        0.40909091,  0.33333333, -0.1       ,  0.4       ,  0.33333333,
        0.2875    ,  0.52222222,  0.        ,  0.4       ,  0.39444444,
        0.08      ,  0.2375    ,  0.45      ,  0.4       ,  0.26151515,
        0.41111111,  0.125     ,  0.35      ,  0.33333333,  0.        ,
        0.37777778,  0.34768519,  0.2825    ,  0.45      ,  0.43333333,
        0.175     , -0.12916667,  0.3       ,  0.3375    ,  0.15      ,
        0.55      ,  0.43888889, -0.01704545,  0.37      ,  0.47777778,
        0.4       ,  0.2625    ,  0.4125    ,  0.3       , -0.2 

In [109]:
r = (max(y_train) - min(y_train)) / 4
m = min(y_train)

In [110]:
y_train = np.array(y_train)

In [113]:
y = []
for i in range(len(y_train)):
    y.append(y_train[i])

In [116]:
y

[0.35,
 0.43333333333333335,
 0.575,
 0.5599999999999999,
 0.6,
 0.24444444444444446,
 0.4,
 0.125,
 0.3499999999999999,
 0.43333333333333335,
 0.5,
 0.27,
 0.3111111111111111,
 0.4121212121212121,
 0.37878787878787873,
 0.4,
 0.6,
 0.16666666666666666,
 0.08952380952380953,
 0.2,
 0.3333333333333333,
 0.0,
 -0.125,
 0.4,
 0.0,
 0.2041666666666667,
 -0.11666666666666665,
 0.3041666666666667,
 0.0,
 0.4,
 0.40909090909090906,
 0.3333333333333333,
 -0.09999999999999994,
 0.4,
 0.3333333333333333,
 0.28750000000000003,
 0.5222222222222223,
 0.0,
 0.4,
 0.39444444444444443,
 0.08,
 0.23750000000000004,
 0.44999999999999996,
 0.4,
 0.26151515151515153,
 0.41111111111111115,
 0.125,
 0.35,
 0.3333333333333333,
 0.0,
 0.37777777777777777,
 0.3476851851851852,
 0.28250000000000003,
 0.44999999999999996,
 0.43333333333333335,
 0.175,
 -0.12916666666666668,
 0.3,
 0.3375,
 0.15,
 0.55,
 0.4388888888888889,
 -0.017045454545454548,
 0.37,
 0.4777777777777778,
 0.39999999999999997,
 0.2625,
 0.4125

In [117]:
X = X_train

In [118]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [119]:
for i in range(len(y)):
    if(y[i] <= m):
        y[i] = 'Negative'
    elif(y[i] <= m+r and y[i] > m):
        y[i] = 'Negative'
    elif(y[i] <= m+r+r and y[i] > m+r):
        y[i] = 'Negative'
    elif(y[i] <= m+r+r+r and y[i] > m+r+r):
        y[i] = 'Positive'
    elif(y[i] <= m+r+r+r+r and y[i] > m+r+r+r):
        y[i] = 'Positive'

In [120]:
y_set = defaultdict(list)


key = average_score.keys()
key = list(key)

In [121]:
for i in range(len(key)):
    y_set[key[i]].append(y[i])

In [122]:
da['calculated rating'] = [y_set[i] for i in y_set.keys()]

In [123]:
rev_rate = []
rate = dataset['rating']
for i in y_set.keys():
    rev_rate.append(rate[i])

In [124]:
for i in range(len(rev_rate)):
    if(rev_rate[i] == 1.0):
        rev_rate[i] = 'Negative'
    elif(rev_rate[i] == 2.0):
        rev_rate[i] = 'Negative'
    elif(rev_rate[i] == 3.0):
        rev_rate[i] = 'Negative'
    elif(rev_rate[i] == 4.0):
        rev_rate[i] = 'Positive'
    elif(rev_rate[i] == 5.0):
        rev_rate[i] = 'Positive'

In [125]:
i

299

In [126]:
list_tuple_word_score = []
for i in range(len(tuple_word_score)):
    list_tuple_word_score.append(tuple_word_score[i])

In [128]:
comparision_dataframe = pd.DataFrame()

In [129]:
comparision_dataframe['Original Review'] = dataset['reviewText']
comparision_dataframe['Opinion Words'] = list_Opinion_Words
comparision_dataframe['Maximum scored Opinion Words'] = list_max_abs_score
comparision_dataframe['Final Opnion Words'] = list_tuple_word_score
comparision_dataframe['Average'] = list_average_score
comparision_dataframe['Reviewer Rating'] = rev_rate
comparision_dataframe['Calculated rating'] = y


In [135]:
#saving the file 

save = comparision_dataframe.to_csv(sep=',')
text_file = open("RED_training_file.csv", "w")
text_file.write(save)
text_file.close()

In [131]:
# Making the Confusion Matrix

y = comparision_dataframe['Calculated rating']

from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(y_test, y_pred)
cm = confusion_matrix(rev_rate, y)

In [132]:
accuracy = 0
for i in range(len(cm)):
    for j in range(len(cm)):
        if(i == j):
            accuracy = accuracy + cm[i][j]
accuracy = accuracy/ len(rev_rate) * 100

In [133]:
TP = np.diag(cm)
FP = np.sum(cm, axis=0) - TP
FN = np.sum(cm, axis=1) - TP

In [134]:
num_classes = len(cm)
TN = []
for i in range(num_classes):
    temp = np.delete(cm, i, 0)    # delete ith row
    temp = np.delete(temp, i, 1)  # delete ith column
    TN.append(sum(sum(temp)))

In [136]:
l = len(rev_rate)
for i in range(num_classes):
    print(TP[i] + FP[i] + FN[i] + TN[i] == l)

True
True


In [137]:
precision = TP/(TP+FP)
recall = TP/(TP+FN)
FScore = 2*(recall * precision) / (recall + precision)

DataFrame = pd.DataFrame()
DataFrame['precision'] = precision
DataFrame['recall'] = recall
DataFrame['FScore'] = FScore

In [159]:
#Testing

New_sentence = "phone is gifted to me by one of best friend Siva Nadar.oh my god!!. i am not much impreseed by the sound quality and touch quality. It isn't the one i can expected. i am disappointed by its short term life span."
New_rating = 'Negative'
New_sentence = ' This is a wonderful screen-touch!! can it get better than this? i am soo much excited!!'
New_rating = 'Positive'

In [160]:
#step 1: pre processing

# isn't = is not
New_sentence = decontracted(New_sentence)


In [161]:
 #removed Unwanted symbols, lower case
comment_dict2 = defaultdict(list)
for i in range(1):
    sentence = re.sub('[^a-zA-Z.]',' ', New_sentence)
    sentence = sentence.lower()
    sentence = sentence.split('.')
    for k in range(len(sentence)):
        review = sentence[k].split()
        review = [word for word in review if not word in set(stopwords.words('english'))]
        sentence[k] =  ' '.join(review)
        comment_dict2[i].append(sentence[k])

In [162]:
#delete unwanted '' words
for j in range(len(comment_dict2)):
    comment_dict2[j] = [comment_dict2[j][i] for i in range(len(comment_dict2[j])) if comment_dict2[j][i] not in '']

In [163]:
Text = defaultdict(list)
for i in range(len(comment_dict2)):
    Text[i].append(('. '.join(comment_dict2[i][j] for j in range(len(comment_dict2[i])))))

In [164]:
# spelling correction
for i in range(len(Text)):
    b = TextBlob(Text[i][0])
    Text[i] = b.correct()

In [165]:
# creating corpus
corpus2 = defaultdict(set)
for i in range(len(Text)):
    wiki = Text[i]
    corpus2[i] = wiki.sentences

In [166]:
corpus_key2 = corpus2.keys()
corpus_list2 = defaultdict(list)

In [167]:
for i in corpus_key2:
    for j in range(len(corpus2[i])):
        word = ' '.join(corpus2[i][j].words)
        corpus_list2[i].append(word)


In [168]:
i

0

In [169]:
Text = ""
for i in range(len(corpus_list2)):
    temp = corpus_list2[i]
    for j in temp:
        Text += str(j) + ' '

In [170]:
Text

'wonderful screen touch get better so much excited '

In [171]:
#  Use OW and find its score
        
list_max_abs_score = []
for i in range(len(max_abs_score)):
    list_max_abs_score.append(max_abs_score[i])

In [172]:
i

299

In [173]:
t = TextBlob(Text)
OW_in_corpus22 = defaultdict(set)
for i in range(len(corpus2)):
    for j in range(len(corpus2[i])):
        word = t.words
        for k in range(len(word)):
            if( word[k] in New_OW):
              OW_in_corpus22[i].add(word[k])
              

In [174]:
t

TextBlob("wonderful screen touch get better so much excited ")

In [175]:
for i in OW_in_corpus22.keys():
    OW_in_corpus22[i] = list(OW_in_corpus22[i])  

In [176]:
testimonial_sentiment = defaultdict(list)
testimonial_sentiment_polarity = defaultdict(list)
OW_in_corpus2 = OW_in_corpus22

In [177]:
for i in OW_in_corpus2.keys():
    for j in range(len(OW_in_corpus2[i])):
        testimonial = TextBlob(OW_in_corpus2[i][j])
        testimonial_sentiment[OW_in_corpus2[i][j]].append(testimonial.sentiment)
        testimonial_sentiment_polarity[OW_in_corpus2[i][j]].append(testimonial.sentiment.polarity)

In [178]:
OW_in_corpus_list = defaultdict(list)
OW_in_corpus_value = defaultdict(list)
for i in OW_in_corpus2.keys():
    for j in range(len(OW_in_corpus2[i])):
        word = [OW_in_corpus2[i][j]]
#        for k in range(len(word)):
        if(word[0] in testimonial_sentiment_polarity.keys()):
            polarity = testimonial_sentiment_polarity[word[0]]
            OW_in_corpus_value[i].append(polarity[0])
            OW_in_corpus_list[i].append(word[0])

In [179]:
dictionary1 = dict()
dictionary2 = dict()
key_value_pair = defaultdict(list)
for i in range(len(OW_in_corpus_list)):
    for j in range(len(OW_in_corpus_list[i])):
#        dictionary1[OW_in_corpus_value[i][j]] = OW_in_corpus_list[i][j]
        dictionary2[ OW_in_corpus_list[i][j]] = OW_in_corpus_value[i][j]  

In [180]:
import numpy as np            
score = defaultdict(list)
words = OW_in_corpus_list
OW_in_corpus_value_key = OW_in_corpus_value.keys()
for i in OW_in_corpus_value_key:
    for j in range(len(OW_in_corpus_value[i])):
        score[i].append(OW_in_corpus_value[i][j])

In [181]:
tuple_word_score = defaultdict(list)
for i in OW_in_corpus_value_key:
    for j in range(len(OW_in_corpus_value[i])):
        tuple_word_score[i].append((words[i][j], score[i][j]))

In [182]:
final_score = defaultdict(list)
for i in range(len(tuple_word_score)):
    final_score[i].append(0)

In [183]:
for i in range(len(tuple_word_score)):
    for j in range(len(tuple_word_score[i])):
        final_score[i].append(tuple_word_score[i][j][1])
        
final_score = defaultdict(list)
for i in range(len(tuple_word_score)):
    final_score[i].append(0)
    

In [187]:
final_score

defaultdict(list, {0: [0, 0.2, 0.5]})

In [184]:
for i in range(len(tuple_word_score)):
    for j in range(len(tuple_word_score[i])):
        final_score[i].append(tuple_word_score[i][j][1])

average_score = defaultdict(list)
for i in range(len(final_score)):
    average_score[i].append(np.mean(final_score[i]))

In [185]:
average_score

defaultdict(list, {0: [0.2333333333333333]})

In [188]:
for i in range(len(average_score)):
    if(np.isnan(average_score[i]) == True):
        average_score[i] = [0]

list_avg = []
for i in range(len(average_score)):
    list_avg.append(average_score[i][0])

In [189]:
list_avg


[0.2333333333333333]

In [190]:
list_tuple_word_score = []
for i in range(len(tuple_word_score)):
    list_tuple_word_score.append(tuple_word_score[i])

In [191]:
y_train = []
for i in average_score.keys():
    y_train.append(average_score[i][0])    

y_train = np.array(y_train)

In [192]:
y = []
for i in range(len(y_train)):
    y.append(y_train[i])

X = X_train

In [193]:
y

[0.2333333333333333]

In [194]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [195]:
for i in range(len(y)):
    if(y[i] <= m):
        y[i] = 'Negative'
    elif(y[i] <= m+r and y[i] > m):
        y[i] = 'Negative'
    elif(y[i] <= m+r+r and y[i] > m+r):
        y[i] = 'Negative'
    elif(y[i] <= m+r+r+r and y[i] > m+r+r):
        y[i] = 'Positive'
    elif(y[i] <= m+r+r+r+r and y[i] > m+r+r+r):
        y[i] = 'Positive'
    

In [196]:
c_dataframe = pd.DataFrame()
c_dataframe['Original Testing Review'] = [New_sentence]
c_dataframe['Processed Testing Review'] = [Text]
c_dataframe['Opinion Words Obtained'] = list_tuple_word_score
c_dataframe['Average of Opinion Words'] = average_score[0]
c_dataframe['Original Rating'] = New_rating
c_dataframe['Predicted Rating'] = y

In [200]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [201]:
X = X_train
y = comparision_dataframe['Calculated rating']
# Splitting the dataset into the Training set and Test set
import sklearn
#from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)



In [202]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [203]:
sc

StandardScaler(copy=True, with_mean=True, with_std=True)

In [204]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 5)

# Fitting DecisionTreeClassifier to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier( criterion = 'entropy', random_state = 10)

In [206]:
classifier = DecisionTreeClassifier( criterion = 'entropy', random_state = 10)

In [207]:
classifier = MultinomialNB()

In [209]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier( n_estimators = 10, criterion = 'entropy', random_state = 10, n_jobs = -1)

In [210]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)

In [212]:
from sklearn.svm import SVC
classifier = SVC(kernel= 'sigmoid', random_state = 10)
classifier = SVC(kernel= 'rbf', random_state = 5)
classifier = SVC(kernel= 'poly', random_state = 0)
classifier = SVC(kernel= 'linear', random_state = 10)

In [213]:
classifier.fit(X_train, y_train)
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [214]:
accuracy = 0
for i in range(len(cm)):
    for j in range(len(cm)):
        if(i == j):
            accuracy = accuracy + cm[i][j]
accuracy = accuracy/ len(y_test) * 100


In [215]:
# 10 fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv=10, n_jobs = -1)

In [216]:
accuracy_by_10fold_cv = (sum(accuracies) / 10)
accuracy_by_10fold_cv = accuracies.mean() * 100
std = accuracies.std() * 100
accuracy = accuracy_by_10fold_cv

In [217]:
TP = np.diag(cm)
FP = np.sum(cm, axis=0) - TP
FN = np.sum(cm, axis=1) - TP

In [218]:
num_classes = len(cm)
TN = []
for i in range(num_classes):
    temp = np.delete(cm, i, 0)    # delete ith row
    temp = np.delete(temp, i, 1)  # delete ith column
    TN.append(sum(sum(temp)))

In [219]:
l = len(y_test)
for i in range(num_classes):
    print(TP[i] + FP[i] + FN[i] + TN[i] == l)

True
True


In [220]:
precision = TP/(TP+FP)
recall = TP/(TP+FN)
FScore = 2*(recall * precision) / (recall + precision)

In [221]:
precision

array([0.66666667, 0.7962963 ])

In [222]:
recall

array([0.56, 0.86])

In [223]:
FScore

array([0.60869565, 0.82692308])

In [224]:
DataFrame = pd.DataFrame()
DataFrame['precision'] = precision
DataFrame['recall'] = recall
DataFrame['FScore'] = FScore

In [225]:
DataFrame

precision  recall    FScore
0   0.666667    0.56  0.608696
1   0.796296    0.86  0.826923

In [226]:
print('\nDataFrame:\n', DataFrame)
print('\n\nConfusion Matrix:\n', cm)
print('\n\nAccuracy: ', accuracy)


DataFrame:
    precision  recall    FScore
0   0.666667    0.56  0.608696
1   0.796296    0.86  0.826923


Confusion Matrix:
 [[14 11]
 [ 7 43]]


Accuracy:  79.10126105778279


In [227]:
print('\n\nOpinionWords:\n', OW_in_corpus)



OpinionWords:
 defaultdict(<class 'set'>, {0: ['product', 'rounded', 'like', 'look', 'popping', 'shape', 'buy', 'stick', 'good'], 1: ['stylish', 'work', 'like', 'great', 'stay', 'stick', 'review', 'share'], 2: ['stylish', 'make', 'almost', 'awesome', 'look', 'quality', 'great', 'believe', 'used'], 3: ['condition', 'big', 'deal', 'screen', 'great', 'claim', 'received', 'perfect', 'free', 'nice', 'get', 'ordered'], 4: ['awesome', 'elevated', 'great', 'multiple', 'used'], 5: ['make', 'purchase', 'like', 'worth', 'price', 'button', 'easy', 'well'], 6: ['ask', 'come', 'people', 'great', 'cut', 'got', 'came'], 7: ['charge', 'waste', 'first'], 8: ['full', 'access', 'end', 'leave', 'solid', 'start', 'build', 'return', 'good'], 9: ['stylish', 'access', 'fantastic', 'double', 'great', 'promised', 'buy', 'easy'], 10: ['complain', 'perfectly'], 11: ['much', 'level', 'slim', 'charge', 'add', 'first', 'handy', 'indicate', 'weight', 'worth', 'price', 'well', 'switch', 'galaxy', 'came'], 12: ['perfo

In [231]:
OW_in_corpus = pd.DataFrame()
OW_in_corpus

Empty DataFrame
Columns: []
Index: []

Empty DataFrame
Columns: []
Index: []

In [236]:
#reading the csvfile for opinion
#for smple file checking
df = pd.read_csv('RED_training_file.csv')
df

Unnamed: 0                                    Original Review  \
0             0  They look good and stick good! I just do not l...   
1             1  These stickers work like the review says they ...   
2             2  These are awesome and make my phone look so st...   
3             3  Item arrived in great time and was in perfect ...   
4             4  awesome! stays on, and looks great. can be use...   
..          ...                                                ...   
295         295  works great for powering my Plantronics BT. i ...   
296         296                                               good   
297         297  I was a bit wary about buying this headset in ...   
298         298  I love that this headset stays firmly on the h...   
299         299  I have been using one of these for years!  I r...   

                                         Opinion Words  \
0    [('product', 0.0), ('rounded', 0.0), ('like', ...   
1    [('stylish', 0.5), ('work', 0.0), ('like', 0.0...   
2    [('stylish', 0.5), ('make', 0.0), ('almost', 0...   
3    [('condition', 0.0), ('big', 0.0), ('deal', 0....   
4    [('awesome', 1.0), ('elevated', 0.0), ('great'...   
..                                                 ...   
295  [('intractable', 0.0), ('long', -0.05), ('way'...   
296                                    [('good', 0.7)]   
297  [('bit', 0.0), ('condition', 0.0), ('tag', 0.0...   
298  [('make', 0.0), ('reason', 0.0), ('sound', 0.4...   
299  [('best', 1.0), ('pick', 0.0), ('hear', 0.0), ...   

    Maximum scored Opinion Words  \
0                [('good', 0.7)]   
1               [('great', 0.8)]   
2             [('awesome', 1.0)]   
3             [('perfect', 1.0)]   
4             [('awesome', 1.0)]   
..                           ...   
295             [('great', 0.8)]   
296              [('good', 0.7)]   
297         [('excellent', 1.0)]   
298             [('great', 0.8)]   
299              [('best', 1.0)]   

                                    Final Opnion Words                Average  \
0                                      [('good', 0.7)]                 [0.35]   
1                   [('stylish', 0.5), ('great', 0.8)]  [0.43333333333333335]   
2    [('stylish', 0.5), ('great', 0.8), ('awesome',...                [0.575]   
3    [('great', 0.8), ('nice', 0.6), ('perfect', 1....   [0.5599999999999999]   
4                   [('great', 0.8), ('awesome', 1.0)]                  [0.6]   
..                                                 ...                    ...   
295  [('easy', 0.43333333333333335), ('great', 0.8)...   [0.3566666666666667]   
296                                    [('good', 0.7)]                 [0.35]   
297  [('great', 0.8), ('new', 0.13636363636363635),...   [0.4840909090909091]   
298  [('nice', 0.6), ('great', 0.8), ('love', 0.5),...   [0.5166666666666666]   
299  [('best', 1.0), ('strong', 0.4333333333333333)...  [0.41393939393939394]   

    Reviewer Rating Calculated rating  
0          Positive          Positive  
1          Positive          Positive  
2          Positive          Positive  
3          Positive          Positive  
4          Positive          Positive  
..              ...               ...  
295        Positive          Positive  
296        Negative          Positive  
297        Positive          Positive  
298        Positive          Positive  
299        Positive          Positive  

[300 rows x 8 columns]

In [234]:
df.head(10)

Unnamed: 0                                    Original Review  \
0           0  They look good and stick good! I just do not l...   
1           1  These stickers work like the review says they ...   
2           2  These are awesome and make my phone look so st...   
3           3  Item arrived in great time and was in perfect ...   
4           4  awesome! stays on, and looks great. can be use...   
5           5  These make using the home button easy. My daug...   
6           6  Came just as described.. It does not come unst...   
7           7  it worked for the first week then it only char...   
8           8  Good case, solid build. Protects phone all aro...   
9           9  This is a fantastic case. Very stylish and pro...   

                                       Opinion Words  \
0  [('product', 0.0), ('rounded', 0.0), ('like', ...   
1  [('stylish', 0.5), ('work', 0.0), ('like', 0.0...   
2  [('stylish', 0.5), ('make', 0.0), ('almost', 0...   
3  [('condition', 0.0), ('big', 0.0), ('deal', 0....   
4  [('awesome', 1.0), ('elevated', 0.0), ('great'...   
5  [('make', 0.0), ('purchase', 0.0), ('like', 0....   
6  [('ask', 0.0), ('come', 0.0), ('people', 0.0),...   
7  [('charge', 0.0), ('waste', -0.2), ('first', 0...   
8  [('full', 0.35), ('access', 0.0), ('end', 0.0)...   
9  [('stylish', 0.5), ('access', 0.0), ('fantasti...   

      Maximum scored Opinion Words  \
0                  [('good', 0.7)]   
1                 [('great', 0.8)]   
2               [('awesome', 1.0)]   
3               [('perfect', 1.0)]   
4               [('awesome', 1.0)]   
5  [('easy', 0.43333333333333335)]   
6                 [('great', 0.8)]   
7                [('first', 0.25)]   
8                  [('good', 0.7)]   
9                 [('great', 0.8)]   

                                  Final Opnion Words                Average  \
0                                    [('good', 0.7)]                 [0.35]   
1                 [('stylish', 0.5), ('great', 0.8)]  [0.43333333333333335]   
2  [('stylish', 0.5), ('great', 0.8), ('awesome',...                [0.575]   
3  [('great', 0.8), ('nice', 0.6), ('perfect', 1....   [0.5599999999999999]   
4                 [('great', 0.8), ('awesome', 1.0)]                  [0.6]   
5    [('easy', 0.43333333333333335), ('worth', 0.3)]  [0.24444444444444446]   
6                                   [('great', 0.8)]                  [0.4]   
7                                  [('first', 0.25)]                [0.125]   
8                    [('full', 0.35), ('good', 0.7)]   [0.3499999999999999]   
9  [('stylish', 0.5), ('easy', 0.4333333333333333...  [0.43333333333333335]   

  Reviewer Rating Calculated rating  
0        Positive          Positive  
1        Positive          Positive  
2        Positive          Positive  
3        Positive          Positive  
4        Positive          Positive  
5        Negative          Positive  
6        Positive          Positive  
7        Negative          Negative  
8        Positive          Positive  
9        Positive          Positive

In [235]:
df.tail(20)

Unnamed: 0                                    Original Review  \
280         280  it works and it works well. i go through charg...   
281         281  Thanks for the quick delivery time. And, it ac...   
282         282  I love that this has a longer cord!  It fits g...   
283         283  Paid $0.32 for the charger and $2.81 Shipping ...   
284         284  Gave this 4 of 5 stars because the cord is waa...   
285         285  Its a charger and it works as advertised. It a...   
286         286  U can never have too many.  Bought this as a s...   
287         287  Product came really fast, nice long cord, whic...   
288         288  Have this in my office at work for a little ex...   
289         289  Bought as gift for my relative and he loves it...   
290         290  Where you mainly use your blue tooth is when y...   
291         291  I have used this car charger for the last 2 mo...   
292         292  it was the wrong one.The male end does not fit...   
293         293  I have not used this as of yet , however, my w...   
294         294  THIS IS A BAD CHARGER IT DOES NOT WORK PROPERL...   
295         295  works great for powering my Plantronics BT. i ...   
296         296                                               good   
297         297  I was a bit wary about buying this headset in ...   
298         298  I love that this headset stays firmly on the h...   
299         299  I have been using one of these for years!  I r...   

                                         Opinion Words  \
280  [('go', 0.0), ('quickly', 0.3333333333333333),...   
281  [('work', 0.0), ('awesome', 1.0), ('pad', 0.0)...   
282  [('great', 0.8), ('pleased', 0.5), ('love', 0....   
283  [('much', 0.2), ('fit', 0.4), ('write', 0.0), ...   
284  [('quickly', 0.3333333333333333), ('buy', 0.0)...   
285  [('can', 0.0), ('quickly', 0.3333333333333333)...   
286  [('worry', 0.0), ('work', 0.0), ('many', 0.5),...   
287  [('long', -0.05), ('product', 0.0), ('fast', 0...   
288  [('needed', 0.0), ('work', 0.0), ('little', -0...   
289  [('bought', 0.0), ('product', 0.0), ('gift', 0...   
290  [('wish', 0.0), ('plug', 0.0), ('handy', 0.6),...   
291  [('intractable', 0.0), ('push', 0.0), ('charge...   
292  [('end', 0.0), ('retract', 0.0), ('fit', 0.4),...   
293  [('issue', 0.0), ('yet', 0.0), ('well', 0.0), ...   
294  [('work', 0.0), ('bad', -0.6999999999999998), ...   
295  [('intractable', 0.0), ('long', -0.05), ('way'...   
296                                    [('good', 0.7)]   
297  [('bit', 0.0), ('condition', 0.0), ('tag', 0.0...   
298  [('make', 0.0), ('reason', 0.0), ('sound', 0.4...   
299  [('best', 1.0), ('pick', 0.0), ('hear', 0.0), ...   

          Maximum scored Opinion Words  \
280                    [('good', 0.7)]   
281                 [('awesome', 1.0)]   
282                   [('great', 0.8)]   
283                   [('great', 0.8)]   
284  [('quickly', 0.3333333333333333)]   
285               [('satisfied', 0.5)]   
286                    [('many', 0.5)]   
287                   [('great', 0.8)]   
288               [('little', 0.1875)]   
289                   [('great', 0.8)]   
290                   [('handy', 0.6)]   
291                    [('nice', 0.6)]   
292                   [('wrong', 0.5)]   
293                  [('really', 0.2)]   
294      [('bad', 0.6999999999999998)]   
295                   [('great', 0.8)]   
296                    [('good', 0.7)]   
297               [('excellent', 1.0)]   
298                   [('great', 0.8)]   
299                    [('best', 1.0)]   

                                    Final Opnion Words  \
280  [('good', 0.7), ('quickly', 0.3333333333333333...   
281                                 [('awesome', 1.0)]   
282  [('great', 0.8), ('pleased', 0.5), ('love', 0....   
283  [('much', 0.2), ('fit', 0.4), ('great', 0.8), ...   
284                  [('quickly', 0.3333333333333333)]   
285  [('satisfied', 0.5), ('quickly', 0.33333333333...   
286                       

In [238]:
#Lets save the above model
from sklearn.externals import joblib

#Save the model as pickle in a file
joblib.dump(precision,'sentiment.pkl')

['sentiment.pkl']